# Applied Data Science Capstone Project W3 Part 2: Add geographical coordinates

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

<ol>
    <li>Start by creating a new Notebook for this assignment.</li>
    <li>Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe </li>
</ol>


## Create the dataframe

<ul>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned</li>
    <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.</li>
    <li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</li>
    <li>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</li>
</ul>

In [1]:
import pandas as pd

In [2]:
url_wp_toronto ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

print("Downloading Data: " + url_wp_toronto)
tables = pd.read_html(url_wp_toronto) # Returns list of all tables on page
toronto_neighborhoods = tables[0]

print("{} records found".format(toronto_neighborhoods.shape[0]))
      
toronto_neighborhoods.head()

288 records found


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
# Check boroughs
toronto_neighborhoods.groupby('Borough').count()

,Postcode,Neighbourhood
Borough,,
Central Toronto,17,17
Downtown Toronto,37,37
East Toronto,7,7
East York,6,6
Etobicoke,45,45
Mississauga,1,1
North York,38,38
Not assigned,77,77
Queen's Park,1,1


### Step 1
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [4]:
toronto_neighborhoods = toronto_neighborhoods[toronto_neighborhoods['Borough'] != 'Not assigned'].reset_index()

### Step 2
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [5]:
toronto_neighborhoods.loc[toronto_neighborhoods['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = toronto_neighborhoods['Borough']

### Step 3
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined in to one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
toronto_agg_neighborhoods = toronto_neighborhoods.groupby(['Postcode','Borough'],as_index=False)['Neighbourhood'].agg(','.join)

In [7]:
print('Cleaning complete!')

Cleaning complete!


In [8]:
print("{} records remaining after data cleaning and shaping".format(toronto_agg_neighborhoods.shape[0]))
toronto_agg_neighborhoods.head()

103 records remaining after data cleaning and shaping


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Step 4
Read the geographical coordinates of each postal code from http://cocl.us/Geospatial_data and merge them in toronto_agg_neighborhoods

In [9]:
url_zip_pos = 'https://cocl.us/Geospatial_data'

toronto_zip_pos = pd.read_csv(url_zip_pos)

In [10]:
toronto_zip_pos.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
toronto_agg_neighborhoods = toronto_agg_neighborhoods.join(toronto_zip_pos.set_index('Postal Code'), on='Postcode')

toronto_agg_neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
